In [33]:
import sklearn
import numpy as np
import importlib
import sklearn.semi_supervised as semi
import pandas as pd
import pickle
import random
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score
from sklearn.model_selection import ParameterGrid
from scipy import sparse
from scipy.spatial import distance
from scipy.sparse import csgraph

In [11]:
pre_data = "./preprocessed/"
data = "./data/"

In [53]:
val = "citeseer"

In [59]:
x = np.load(pre_data + val +'_features.npy')

with open(data + 'ind.' + val + '.y', 'rb') as pickle_file:
    tr_y = pickle.load(pickle_file)

with open(data + 'ind.' + val + '.ally', 'rb') as pickle_file:
    tr_all_y = pickle.load(pickle_file)
    
with open(data + 'ind.' + val + '.ty', 'rb') as pickle_file:
    t_y = pickle.load(pickle_file)

In [68]:
y = np.concatenate([tr_all_y, t_y])
x = x[:y.shape[0]]

(3312, 3703)

In [56]:
# Citeseer dictionary has some values in dict that dont exist as node
if val == "citeseer":
    adj_dict = {k: list(filter(lambda j: j < x.shape[0], v)) for k, v in adj_dict.items() if k < x.shape[0]}

NameError: name 'adj_dict' is not defined

In [655]:
adj_dict

{0: [628],
 1: [158, 2919, 2933, 1097, 486],
 2: [3285],
 3: [3219, 1431],
 4: [467],
 5: [648],
 6: [1501],
 7: [2137, 1833],
 8: [178, 1033],
 9: [1007],
 10: [2622, 1670],
 11: [2034],
 12: [2487, 1622, 794, 966, 113, 557, 677, 1357, 1097, 2474, 839, 1760],
 13: [1894, 2711, 1493, 1871, 1167],
 14: [1248, 146],
 15: [2521],
 16: [2415,
  314,
  2416,
  616,
  1013,
  2417,
  2418,
  2166,
  1422,
  2597,
  1567,
  622,
  1605,
  1708],
 17: [879, 2150],
 18: [812, 582, 778],
 19: [1451, 1338, 3308],
 20: [3080],
 21: [1073],
 22: [726, 2140, 3251],
 23: [1942],
 24: [1227, 2027],
 25: [2057, 2331],
 26: [2729, 222],
 27: [755, 1290, 2782, 1422, 1943, 229, 1012, 2021, 2023, 2282],
 28: [28, 28, 2312, 1578, 1447, 2313, 2314, 2315, 1678, 2316, 1912, 135, 2059],
 29: [1422, 2838],
 30: [1422, 532],
 31: [136, 1759, 1131, 1625],
 32: [118, 3009],
 33: [3193, 1822, 824, 3197],
 34: [75],
 35: [89],
 36: [763],
 37: [1745],
 38: [2388, 2228],
 39: [1412, 2466],
 40: [631, 2625],
 41: [606,

In [69]:
y_flat = np.argmax(y, axis = 1).flatten()

In [73]:
x.shape

(3312, 3703)

In [71]:
n_samples = x.shape[0]
indices = np.arange(n_samples)
if val == "cora":
    n_labeled_points = 140
elif val == "citeseer":
    n_labeled_points = 120

# They take first n_labeled_points and unlabel the rest
unlabeled_set = indices[n_labeled_points:]

y_train = np.copy(y_flat)
y_train[unlabeled_set] = -1

In [648]:
rbf_grid = ParameterGrid({
    'alpha': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'gamma': [0.01, 0.1, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
})

knn_grid = ParameterGrid({
    'alpha': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13]
})

lp_grid = ParameterGrid({
    'alpha': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'sig': [0.01, 0.1, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 10000]
})

In [546]:
rbf = pd.DataFrame({"Kernel": [], "Alpha": [], 
                    "Gamma": [], "Accuracy": [],
                    "Prec": [], "Recall": [], "f1": [],
                    "W Avg. Prec": [], "W. Avg. Recall": [], "W. Avg. f1": []}, 
             columns=["Kernel", "Alpha", "Gamma", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"])
true_labels = y_flat[unlabeled_set]

for params in list(rbf_grid):
    lp = semi.LabelSpreading(**params, max_iter=500)
    lp.fit(x, y_train)
    predicted_labels = lp.transduction_[unlabeled_set]
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    w_avg = report["weighted avg"]
    w_mic = report["micro avg"]
    
    
    one_run = pd.DataFrame({"Kernel": "rbf", "Alpha": params["alpha"], 
                            "Gamma": params["gamma"], "Accuracy": acc,
                            "W Avg. Prec": w_avg["precision"], 
                            "W. Avg. Recall": w_avg["recall"], 
                            "W. Avg. f1": w_avg["f1-score"],
                            "Precision": w_mic["precision"], 
                            "Recall": w_mic["recall"], 
                            "f1": w_mic["f1-score"]}, 
             columns=["Kernel", "Alpha", "Gamma", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"], index=[rbf.shape[0]])
    rbf = rbf.append(one_run)
    
rbf.to_csv(val + "_lp_rbf_results.csv")

ValueError: Found input variables with inconsistent numbers of samples: [2708, 3312]

In [496]:
knn = pd.DataFrame({"Kernel": [], "Alpha": [], 
                    "n_neighbors": [], "Accuracy": [], 
                    "Precision": [], "Recall": [], "f1": []}, 
             columns=["Kernel", "Alpha", "n_neighbors", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"])
true_labels = y_flat[unlabeled_set]

for params in list(knn_grid):
    lp = semi.LabelSpreading(**params, max_iter=500)
    lp.fit(x, y_train)
    predicted_labels = lp.transduction_[unlabeled_set]
    
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    w_avg = report["weighted avg"]
    w_mic = report["micro avg"]
    acc = accuracy_score(true_labels, predicted_labels)
    
    
    one_run = pd.DataFrame({"Kernel": "knn", "Alpha": params["alpha"], 
                            "n_neighbors": params["n_neighbors"], 
                            "Accuracy": acc,
                            "W Avg. Prec": w_avg["precision"], 
                            "W. Avg. Recall": w_avg["recall"], 
                            "W. Avg. f1": w_avg["f1-score"],
                            "Precision": w_mic["precision"], 
                            "Recall": w_mic["recall"], 
                            "f1": w_mic["f1-score"]}, 
             columns=["Kernel", "Alpha", "n_neighbors", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"], index=[knn.shape[0]])
    knn = knn.append(one_run)

    
knn.to_csv(val + "_lp_knn_results.csv")

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\s

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value enc

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value enc

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value enc

In [658]:
labelprop = pd.DataFrame({"Kernel": [], "Alpha": [], 
                    "sigma": [], "Accuracy": [], 
                    "Precision": [], "Recall": [], "f1": []}, 
             columns=["Kernel", "Alpha", "sigma", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"])
true_labels = y_flat[unlabeled_set]

for params in list(lp_grid):
    lp = LabelProp(**params, adj_dict = adj_dict)
    lp.fit(x, y_train)
    predicted_labels = lp.transduction_[unlabeled_set]
    
    report = classification_report(true_labels, predicted_labels, output_dict=True)
    w_avg = report["weighted avg"]
    w_mic = report["micro avg"]
    acc = accuracy_score(true_labels, predicted_labels)
    
    
    one_run = pd.DataFrame({"Kernel": "labelprob", "Alpha": params["alpha"], 
                            "sigma": params["sig"], 
                            "Accuracy": acc,
                            "W Avg. Prec": w_avg["precision"], 
                            "W. Avg. Recall": w_avg["recall"], 
                            "W. Avg. f1": w_avg["f1-score"],
                            "Precision": w_mic["precision"], 
                            "Recall": w_mic["recall"], 
                            "f1": w_mic["f1-score"]}, 
             columns=["Kernel", "Alpha", "sigma", 
                      "Accuracy", "Precision",  "Recall", 
                       "f1", "W Avg. Prec", "W. Avg. Recall", 
                      "W. Avg. f1"], index=[labelprop.shape[0]])
    labelprop = labelprop.append(one_run)

    
labelprop.to_csv(val + "_lp_laplacian_results.csv")

[[ -0.         -60.00000339 -54.00000043 ... -61.99999729 -64.
  -60.00000339]
 [-60.00000339  -0.         -58.00000145 ... -64.         -67.99999671
  -64.        ]
 [-54.00000043 -58.00000145  -0.         ... -49.99999997 -58.00000145
  -54.00000043]
 ...
 [-61.99999729 -64.         -49.99999997 ...  -0.         -69.99999617
  -67.99999671]
 [-64.         -67.99999671 -58.00000145 ... -69.99999617  -0.
  -61.99999729]
 [-60.00000339 -64.         -54.00000043 ... -67.99999671 -61.99999729
   -0.        ]]
matrix [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Diag: 
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


C:\Users\Ali\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in power


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[ -0.         -60.00000339 -54.00000043 ... -61.99999729 -64.
  -60.00000339]
 [-60.00000339  -0.         -58.00000145 ... -64.         -67.99999671
  -64.        ]
 [-54.00000043 -58.00000145  -0.         ... -49.99999997 -58.00000145
  -54.00000043]
 ...
 [-61.99999729 -64.         -49.99999997 ...  -0.         -69.99999617
  -67.99999671]
 [-64.         -67.99999671 -58.00000145 ... -69.99999617  -0.
  -61.99999729]
 [-60.00000339 -64.         -54.00000043 ... -67.99999671 -61.99999729
   -0.        ]]


C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer
C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


matrix [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Diag: 
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


...

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


matrix [[1.         0.9999997  0.99999973 ... 0.99999969 0.99999968 0.9999997 ]
 [0.9999997  1.         0.99999971 ... 0.99999968 0.99999966 0.99999968]
 [0.99999973 0.99999971 1.         ... 0.99999975 0.99999971 0.99999973]
 ...
 [0.99999969 0.99999968 0.99999975 ... 1.         0.99999965 0.99999966]
 [0.99999968 0.99999966 0.99999971 ... 0.99999965 1.         0.99999969]
 [0.9999997  0.99999968 0.99999973 ... 0.99999966 0.99999969 1.        ]]
Diag: 
[[3311.99901923    0.            0.         ...    0.
     0.            0.        ]
 [   0.         3311.99897142    0.         ...    0.
     0.            0.        ]
 [   0.            0.         3311.99912059 ...    0.
     0.            0.        ]
 ...
 [   0.            0.            0.         ... 3311.99891942
     0.            0.        ]
 [   0.            0.            0.         ...    0.
  3311.99893188    0.        ]
 [   0.            0.            0.         ...    0.
     0.         3311.99900715]]
[[1.         0.999

C:\Users\Ali\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in power


[[0.0173762 0.        0.        ... 0.        0.        0.       ]
 [0.        0.0173762 0.        ... 0.        0.        0.       ]
 [0.        0.        0.0173762 ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.0173762 0.        0.       ]
 [0.        0.        0.        ... 0.        0.0173762 0.       ]
 [0.        0.        0.        ... 0.        0.        0.0173762]]
[[0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]
 [0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]
 [0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]
 ...
 [0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]
 [0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]
 [0.00030193 0.00030193 0.00030193 ... 0.00030193 0.00030193 0.00030193]]


C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [82]:
true_labels = y_flat[unlabeled_set]
class LabelProp(semi.LabelSpreading):
    def __init__(self, sig = 2, adj_dict = None, kernel='rbf', gamma=100, n_neighbors=3,
                 alpha=0.9, max_iter=1000, tol=1e-10, n_jobs=None):
        super().__init__(kernel, gamma, n_neighbors, alpha, max_iter, tol, n_jobs)
        self.sig = sig
        
    def _get_sim(self, i, k):
        x = self.X_
        x_one = x[i]
        x_two = x[k]
        dist = np.power(distance.euclidean(x_one, x_two), 2)

        # w = np.exp(-dist/(2*np.power((self.sig), 2)))
        return -dist
        
    def _build_matrix(self):
        d_len = self.X_.shape[0]
#         matrix = np.array([[self._get_sim(x, k) if x in v else 0 for x in self.adj_dict.keys()] 
#                             for k, v in self.adj_dict.items()])
#         matrix = np.array([[self._get_sim(i, j) for j in range(d_len)] 
#                            for i in range(d_len)])
        
        
#         matrix = np.zeros((d_len, d_len))
#         for i in range(matrix.shape[0]):
#             for j in range(i, matrix.shape[0]):
#                 sim = self._get_sim(i, j)
#                 matrix[i, j] = sim
#                 matrix[j, i] = sim
#        np.save(val + "_matrix_all_nodes", matrix)
#         matrix = np.array([[self._get_sim(x, k) if x in v else 0 for x in self.adj_dict.keys()] 
#                             for k, v in self.adj_dict.items()])
#         np.save(val + "_matrix_w_graph", matrix)
        adj_matrix = np.load(pre_data + val + "_adj.npy")
        adj_matrix = adj_matrix[:d_len, :d_len]
#         matrix = np.load(val + "_matrix_all_nodes.npy")
#         print(matrix)
#         matrix = np.exp(matrix/(2*np.power(self.sig, 2)))
#         diag = np.zeros((d_len, d_len))
#         vals = matrix.sum(axis=0)/2
#         for i in range(diag.shape[0]):
#             diag[i, i] = matrix[i].sum()
#         print("Diag: ")
#         print(diag)
        
#         print(matrix)
        
#         # Normalize the matrix
#         sq_diag = np.power(diag, -1/2)
#         sq_diag[np.isinf(sq_diag)] = 0
#         print(sq_diag)
        
#         S = np.matmul(np.matmul(sq_diag, matrix), sq_diag)

        
        # print(S)
        
        return adj_matrix

    def _build_graph(self):
        """Matrix representing a fully connected graph between each sample
        This basic implementation creates a non-stochastic affinity matrix, so
        class distributions will exceed 1 (normalization may be desired).
        """
        affinity_matrix = self._build_matrix()
        return affinity_matrix

lp = LabelProp(alpha=0.1)
lp.fit(x, y_train)
predicted_labels = lp.transduction_[unlabeled_set]
predicted_labels

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [83]:
print(classification_report(true_labels, predicted_labels))
print(accuracy_score(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.11      0.61      0.19       229
           1       0.42      0.22      0.29       570
           2       0.52      0.39      0.44       648
           3       0.53      0.22      0.31       681
           4       0.55      0.46      0.50       576
           5       0.43      0.32      0.36       488

   micro avg       0.34      0.34      0.34      3192
   macro avg       0.43      0.37      0.35      3192
weighted avg       0.47      0.34      0.37      3192

0.3408521303258145
